In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4433758481980190841
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5750390784
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3277636562185094230
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
import shutil, os

image_dir = 'D:/python-ai-data/232.재활용품 분류 및 선별 데이터/01.데이터/data/images/'
image_dir_target = 'D:/python-ai-data/232.재활용품 분류 및 선별 데이터/01.데이터/train/images/'
item_image_list = os.listdir(image_dir)

for item in item_image_list:
    shutil.copytree(image_dir+item+'/images0/', image_dir_target, dirs_exist_ok=True)
    shutil.copytree(image_dir+item+'/images1/', image_dir_target, dirs_exist_ok=True)
    shutil.copytree(image_dir+item+'/images2/', image_dir_target, dirs_exist_ok=True)
    shutil.copytree(image_dir+item+'/images3/', image_dir_target, dirs_exist_ok=True)

label_dir = 'D:/python-ai-data/232.재활용품 분류 및 선별 데이터/01.데이터/data/labels/'
label_dir_target = 'D:/python-ai-data/232.재활용품 분류 및 선별 데이터/01.데이터/train/labels_json/'
item_label_list = os.listdir(label_dir)

for item in item_label_list:
    shutil.copytree(label_dir+item+'/labels0/', label_dir_target, dirs_exist_ok=True)
    shutil.copytree(label_dir+item+'/labels1/', label_dir_target, dirs_exist_ok=True)
    shutil.copytree(label_dir+item+'/labels2/', label_dir_target, dirs_exist_ok=True)
    shutil.copytree(label_dir+item+'/labels3/', label_dir_target, dirs_exist_ok=True)

In [6]:
import json
import os
import pandas as pd
import numpy as np

base_dir = 'D:/python-ai-data/232.재활용품 분류 및 선별 데이터/01.데이터'

train_images_path = base_dir + '/train/images/'
train_labels_json_path = base_dir + '/train/labels_json/'
train_labels_path = base_dir + '/train/labels/'

os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_json_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)

# !unzip -qq '/content/drive/MyDrive/KCC3기_2차_프로젝트/train.zip' -d '/content/data/train'

train_images_list = os.listdir(train_images_path)
train_labels_json_list = os.listdir(train_labels_json_path)

train_images_list.sort()
train_labels_json_list.sort()
     

In [10]:
# 실제 train data를 변환하기 - 라벨링 변환

yolo_class = {
    '철캔': 0,
    '알루미늄캔': 4,
    '종이': 8,
    '무색단일': 12,
    '유색단일': 16,
    'PE': 20,
    'PP': 24,
    'PS': 28,
    '스티로폼': 32,
    '비닐': 36,
    '갈색': 40,
    '녹색': 44,
    '투명': 48
}

for label in train_labels_json_list:
    file_json = open(train_labels_json_path + label, encoding='UTF-8')
    f_json = json.load(file_json)

    img_width = f_json['IMAGE_INFO']['IMAGE_WIDTH']
    img_height = f_json['IMAGE_INFO']['IMAGE_HEIGHT']

    f_txt = open(train_labels_path + label.replace('.json', '.txt'), 'w')

    for anno in f_json['ANNOTATION_INFO']:
        obj_id = yolo_class[anno['DETAILS']]

        if anno['DAMAGE'] != '원형':
            obj_id += 2
        if anno['DIRTINESS'] != '오염없음':
            obj_id += 1

        obj_points = np.array(anno['POINTS'])
        if obj_points[0].size == 4:
            yolo_id = obj_id
            yolo_points = np.array([obj_points[0][0]+obj_points[0][2]/2, obj_points[0][1]+obj_points[0][3]/2,
                                    obj_points[0][2], obj_points[0][3]])

            yolo_points_norm = np.array([yolo_points[0]/img_width, yolo_points[1]/img_height,
                             yolo_points[2]/img_width, yolo_points[3]/img_height])

            yolo_points_norm = list(yolo_points_norm)
            f_txt.write(str(yolo_id) + ' ')
            f_txt.write(' '.join(str(ele) for ele in yolo_points_norm))
            f_txt.write('\n')
        elif obj_points.size >= 2:
            max_x = 0
            min_x = img_width
            max_y = 0
            min_y = img_height
            for points in obj_points:
                if points[0] > max_x:
                    max_x = points[0]
                if points[0] < min_x:
                    min_x = points[0]
                if points[1] > max_y:
                    max_y = points[1]
                if points[1] < min_y:
                    min_y = points[1]

            yolo_id = obj_id
            yolo_points = np.array([(min_x+max_x)/2, (min_y+max_y)/2,
                              max_x-min_x, max_y-min_y])
            yolo_points_norm = np.array([yolo_points[0]/img_width, yolo_points[1]/img_height,
                                         yolo_points[2]/img_width, yolo_points[3]/img_height])
            yolo_points_norm = list(yolo_points_norm)

            f_txt.write(str(yolo_id) + ' ')
            f_txt.write(' '.join(str(ele) for ele in yolo_points_norm))
            f_txt.write('\n')
        else:
            print(label)
    f_txt.close()

     

In [ ]:
# json 라벨 삭제 및 라벨링 확인

import shutil

shutil.rmtree(train_labels_json_path)

# print(open('/content/data/train/labels/606511@4_04002_220907_P1_T1.txt').readline())
# print(open('/content/data/train/labels/10349@3_01001_220715_P1_T1.txt').readline())
     

In [11]:
# validation 데이터 분리

import shutil

val_images_path = base_dir + '/val/images/'
val_labels_path = base_dir + '/val/labels/'

os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

train_images_list = os.listdir(train_images_path)
train_labels_list = os.listdir(train_labels_path)

train_images_list.sort()
train_labels_list.sort()

for i in range(len(train_images_list)):
    if i % 10 == 0:
        shutil.move(train_images_path + train_images_list[i], val_images_path)
        shutil.move(train_labels_path + train_labels_list[i], val_labels_path)


In [13]:
# 분리한 데이터 이미지와 라벨 파일 일치 여부 확인

print(len(os.listdir(train_images_path)))
print(len(os.listdir(train_labels_path)))

print(len(os.listdir(val_images_path)))
print(len(os.listdir(val_labels_path)))

train_images_list = os.listdir(train_images_path)
train_labels_list = os.listdir(train_labels_path)

train_images_list.sort()
train_labels_list.sort()

for i in range(len(train_images_list)):
    if train_images_list[i].replace('.jpg', '') != train_labels_list[i].replace('.txt', ''):
        print(train_images_list[i])

val_images_list = os.listdir(val_images_path)
val_labels_list = os.listdir(val_labels_path)

val_images_list.sort()
val_labels_list.sort()

print()
for i in range(len(val_images_list)):
    if val_images_list[i].replace('.jpg', '') != val_labels_list[i].replace('.txt', ''):
        print(val_images_list[i])


77356
77356
8596
8596



In [1]:
!pip install PyYAML

import yaml

data = {
    'train': 'D:/python-ai-data/train/images/',
    'val': 'D:/python-ai-data/val/images/',
    'names': ['IronCan_NoDamNoDir', 'IronCan_NoDamDir', 'IronCan_DamNoDir', 'IronCan_DamDir',
              'AluminumCan_NoDamNoDir', 'AluminumCan_NoDamDir', 'AluminumCan_DamNoDir', 'AluminumCan_DamDir',
              'Paper_NoDamNoDir', 'Paper_NoDamDir', 'Paper_DamNoDir', 'Paper_DamDir',
              'PetNoColor_NoDamNoDir', 'PetNoColor_NoDamDir', 'PetNoColor_DamNoDir', 'PetNoColor_DamDir',
              'PetColor_NoDamNoDir', 'PetColor_NoDamDir', 'PetColor_DamNoDir', 'PetColor_DamDir',
              'PE_NoDamNoDir', 'PE_NoDamDir', 'PE_DamNoDir', 'PE_DamDir',
              'PP_NoDamNoDir', 'PP_NoDamDir', 'PP_DamNoDir', 'PP_DamDir',
              'PS_NoDamNoDir', 'PS_NoDamDir', 'PS_DamNoDir', 'PS_DamDir',
              'Styrofoam_NoDamNoDir', 'Styrofoam_NoDamDir', 'Styrofoam_DamNoDir', 'Styrofoam_DamDir',
              'Vinyl_NoDamNoDir', 'Vinyl_NoDamDir', 'Vinyl_DamNoDir', 'Vinyl_DamDir',
              'GlassBrown_NoDamNoDir', 'GlassBrown_NoDamDir', 'GlassBrown_DamNoDir', 'GlassBrown_DamDir',
              'GlassGreen_NoDamNoDir', 'GlassGreen_NoDamDir', 'GlassGreen_DamNoDir', 'GlassGreen_DamDir',
              'GlassTransparency_NoDamNoDir', 'GlassTransparency_NoDamDir', 'GlassTransparency_DamNoDir', 'GlassTransparency_DamDir'],
    'nc': 52
}

yaml_name = 'recycle_data.yaml'
with open('D:/python-ai-data/' + 'recycle_data.yaml', 'w') as f:
    yaml.dump(data, f)

with open('D:/python-ai-data/' + 'recycle_data.yaml', 'r') as f:
    recycle_yaml = yaml.safe_load(f)
    display(recycle_yaml)
     

{'names': ['IronCan_NoDamNoDir',
  'IronCan_NoDamDir',
  'IronCan_DamNoDir',
  'IronCan_DamDir',
  'AluminumCan_NoDamNoDir',
  'AluminumCan_NoDamDir',
  'AluminumCan_DamNoDir',
  'AluminumCan_DamDir',
  'Paper_NoDamNoDir',
  'Paper_NoDamDir',
  'Paper_DamNoDir',
  'Paper_DamDir',
  'PetNoColor_NoDamNoDir',
  'PetNoColor_NoDamDir',
  'PetNoColor_DamNoDir',
  'PetNoColor_DamDir',
  'PetColor_NoDamNoDir',
  'PetColor_NoDamDir',
  'PetColor_DamNoDir',
  'PetColor_DamDir',
  'PE_NoDamNoDir',
  'PE_NoDamDir',
  'PE_DamNoDir',
  'PE_DamDir',
  'PP_NoDamNoDir',
  'PP_NoDamDir',
  'PP_DamNoDir',
  'PP_DamDir',
  'PS_NoDamNoDir',
  'PS_NoDamDir',
  'PS_DamNoDir',
  'PS_DamDir',
  'Styrofoam_NoDamNoDir',
  'Styrofoam_NoDamDir',
  'Styrofoam_DamNoDir',
  'Styrofoam_DamDir',
  'Vinyl_NoDamNoDir',
  'Vinyl_NoDamDir',
  'Vinyl_DamNoDir',
  'Vinyl_DamDir',
  'GlassBrown_NoDamNoDir',
  'GlassBrown_NoDamDir',
  'GlassBrown_DamNoDir',
  'GlassBrown_DamDir',
  'GlassGreen_NoDamNoDir',
  'GlassGreen_NoDamD

In [2]:

# YOLO 학습 위한 환경

# %pip install ultralytics

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.159  Python-3.8.17 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
Setup complete  (12 CPUs, 31.8 GB RAM, 1336.1/1863.0 GB disk)


In [5]:
# YOLO 전이 학습
from ultralytics import YOLO
import shutil, os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

model = YOLO('yolov8n.pt')

model.train(data='D:/python-ai-data/recycle_data.yaml', epochs=1, patience=3, batch=64, imgsz=640, 
            name='recycle', exist_ok=True, device=0)

# if os.path.isdir('/content/drive/MyDrive/KCC_2nd_project/models/recycle'):
#     shutil.rmtree('/content/drive/MyDrive/KCC_2nd_project/models/recycle')

# shutil.copytree('/content/runs/detect/recycle/' , '/content/drive/MyDrive/KCC_2nd_project/models/recycle/')
     

Ultralytics YOLOv8.0.159  Python-3.8.17 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:/python-ai-data/recycle_data.yaml, epochs=1, patience=3, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=recycle, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optim

     PetColor_DamNoDir       8596         53      0.249      0.264      0.165      0.163
       PetColor_DamDir       8596        106      0.182       0.34      0.199      0.191
         PE_NoDamNoDir       8596         40          1          0     0.0432     0.0429
           PE_NoDamDir       8596        310      0.346      0.735      0.474      0.428
           PE_DamNoDir       8596         17          1          0    0.00606     0.0045
             PE_DamDir       8596         31          1          0     0.0157     0.0154
         PP_NoDamNoDir       8596        300      0.394      0.793      0.543      0.479
           PP_NoDamDir       8596        241      0.267      0.477      0.289      0.243
           PP_DamNoDir       8596         88      0.272      0.261      0.198      0.185
             PP_DamDir       8596         49          0          0     0.0299     0.0276
         PS_NoDamNoDir       8596        211      0.344       0.45      0.313      0.294
           PS_NoDamDi

In [ ]:
# google drive의 모델 파일을 불러들여 다시 전이 학습
# 학습 완료 시 다시 google drive로 모델 파일 이동

from ultralytics import YOLO
import shutil, os

model = YOLO('/content/drive/MyDrive/KCC_2nd_project/models/recycle/weights/best.pt')

model.train(data='/content/data/recycle_data.yaml', epochs=3, patience=3, batch=64, imgsz=640, name='recycle', exist_ok=True)

# if os.path.isdir('/content/drive/MyDrive/KCC_2nd_project/models/recycle'):
#     shutil.rmtree('/content/drive/MyDrive/KCC_2nd_project/models/recycle')

# shutil.copytree('/content/runs/detect/recycle/' , '/content/drive/MyDrive/KCC_2nd_project/models/recycle/')
     

In [ ]:
# 예측 값 확인

from ultralytics import YOLO
import shutil, os
from PIL import Image
import matplotlib.pyplot as plt

model = YOLO('/content/drive/MyDrive/KCC_2nd_project/models/recycle/weights/best.pt')


for i in range(10):
    results = model('/content/data/val/images/' + os.listdir('/content/data/val/images/')[i])

    # Show the results
    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        plt.imshow(im)
        plt.show()
     